In this task I'm going to make a interactive map that show the locations of tram stops in Helsinki. On the other layer there will be a heatmap to explain the distribution and density of the stops.

In [4]:
# Importing modules
import folium
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

In [14]:
# Reading HSL_stops data to GeoDataFrame
fp = r'HSL_stops.shp'
tram_stops = gpd.read_file(fp)

# Changing the tram stops' crs to WGS84 as well
tram_stops = tram_stops.to_crs(epsg=4326)

# Selecting only tram stops from the data
tram_stops = tram_stops.loc[tram_stops["VERKKO"] == 3]

# Selecting only the stops in use
tram_stops = tram_stops.loc[tram_stops["REI_VOIM"] == 1]

# Converting the tram stops to GeoJSON features
tram_stops_gjson = folium.features.GeoJson(tram_stops, name="Tram stops")

,FID,SOLMUTUNNU,LYHYTTUNNU,SOLMUTYYPP,X,Y,PROJ_X,PROJ_Y,NIMI1,NAMN1,NIMI2,NAMN2,REI_VOIM,AIK_VOIM,VERKKO,PYSAKKITYY,geometry
27,28,1010419,H0450,P,2553099,6673549,2553111,6673549,Hallituskatu,Regeringsgatan,Snellmaninkatu 4,Snellmansgatan 4,1,1,3,04,POINT (24.95332 60.17012)
28,29,1010420,H0451,P,2553121,6673575,2553109,6673575,Hallituskatu,Regeringsgatan,Snellmaninkatu 3,Snellmansgatan 3,1,1,3,01,POINT (24.95372 60.17036)
29,30,1010421,H0453,P,2553105,6673774,2553093,6673774,Kansallisarkisto,Riksarkivet,Snellmaninkatu 13,Snellmansgatan 13,1,1,3,01,POINT (24.95348 60.17214)
30,31,1010422,H0452,P,2553083,6673737,2553096,6673737,Kansallisarkisto,Riksarkivet,Snellmaninkatu 8,Snellmansgatan 8,1,1,3,01,POINT (24.95308 60.17181)


In [13]:
# Creating a Map instance, setting OpenStreetMap to background map
m = folium.Map(location=[60.19, 24.93], tiles = 'openstreetmap', zoom_start=11.55, control_scale=True)

# Getting x and y coordinates for each stop
tram_stops["x"] = tram_stops["geometry"].apply(lambda geom: geom.x)
tram_stops["y"] = tram_stops["geometry"].apply(lambda geom: geom.y)

# Creating a list of coordinate pairs
locations = list(zip(tram_stops["y"], tram_stops["x"]))

# Importing Heatmap plugin
from folium.plugins import HeatMap

# Creating a feature group to heatmap and naming it, setting it to not be automatically visible
fg=folium.FeatureGroup(name='Tram stop heatmap', show=False)
HeatMap(locations).add_to(fg)

# Adding heatmap to map instance
fg.add_to(m)

# Creating second feature group for tram stops.
fg2=folium.FeatureGroup(name="Tram stops", show=True)

# Adding the tramp stops to map
tram_stops_gjson.add_to(fg2)

# Modifying the icons indicating tram stops
for tuple_ in locations:
    icon=folium.Icon(color='beige', icon='train', icon_color="green", prefix='fa')
    folium.Marker(tuple_, icon=icon).add_to(fg2)
fg2.add_to(m)

# Creating a layer control object and adding it to map instance
folium.LayerControl().add_to(m)

outfp = f"docs/tram_stops.html"
m.save(outfp)